# Predicción de Tiempos por Vuelta en F1

Este notebook implementa modelos predictivos para estimar el tiempo por vuelta de los coches de F1 en función de diversas variables como el tipo de neumático, condiciones meteorológicas, y estado de la pista.

## Objetivos
1. Cargar y preprocesar datos de FastF1 y OpenF1
2. Realizar feature engineering para potenciar la capacidad predictiva
3. Incluir análisis de degradación de neumáticos y paradas en boxes
4. Entrenar modelos de predicción (XGBoost y opcionalmente una Red Neuronal)
5. Evaluar el rendimiento y visualizar resultados

## 1. Importación de Librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
import joblib
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Configuración de visualización
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Configuración de fastf1
fastf1.Cache.enable_cache('../f1-strategy/f1_cache')  # Asegúrate de que esta carpeta exista

# Crear directorios para outputs y models si no existen
os.makedirs('../outputs/week3', exist_ok=True)
os.makedirs('../models/week3', exist_ok=True)

## 2. Definición de Modelos

In [3]:
# Clase para el modelo PyTorch
class LapTimeNN(nn.Module):
    def __init__(self, input_size):
        super(LapTimeNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    
    def forward(self, x):
        return self.model(x)

## 3. Carga y Preparación de Datos

Vamos a cargar los datos desde los archivos Parquet que tenemos disponibles:
- laps.parquet: Contiene información sobre vueltas individuales
- weather.parquet: Contiene información meteorológica
- intervals.parquet: Contiene información sobre gaps y estados de carrera
- pitstops.parquet: Contiene información sobre paradas en boxes

In [ ]:
# Cargar datos desde archivos parquet
def load_all_data():
    """
    Carga todos los datasets desde archivos parquet y los devuelve como DataFrames
    """
    # Definir rutas a los archivos
    laps_path = "../f1-strategy/data/raw/Spain_2023_laps.parquet"
    weather_path = "../f1-strategy/data/raw/Spain_2023_weather.parquet"
    intervals_path = "../f1-strategy/data/raw/Spain_2023_openf1_intervals.parquet"
    pitstops_path = "../f1-strategy/data/raw/Spain_2023_pitstops.parquet"
    
    # Cargar DataFrames
    laps_df = pd.read_parquet(laps_path)
    weather_df = pd.read_parquet(weather_path)
    

    # Establish booleans of intervals and pitstops to True
    # Before, error magaing to knwo if this dataframes could be correctly downloaded
    
    return laps_df, weather_df, intervals_df, pitstops_df, has_intervals, has_pitstops



In [7]:
# Ejecutar la carga de datos
laps_df, weather_df, intervals_df, pitstops_df, has_intervals, has_pitstops = load_all_data()



In [8]:
# Mostrar información sobre los DataFrames
print("Dimensiones de laps_df:", laps_df.shape)
print("Dimensiones de weather_df:", weather_df.shape)
if has_intervals:
    print("Dimensiones de intervals_df:", intervals_df.shape)
if has_pitstops:
    print("Dimensiones de pitstops_df:", pitstops_df.shape)

Dimensiones de laps_df: (1312, 31)
Dimensiones de weather_df: (154, 8)
Dimensiones de intervals_df: (8933, 10)
Dimensiones de pitstops_df: (43, 31)


## 4. Exploración Inicial de Datos

In [ ]:
# Explorar los primeros registros
print("Primeros registros de laps_df:")
display(laps_df.head())

print("\nPrimeros registros de weather_df:")
display(weather_df.head())

if has_pitstops:
    print("\nPrimeros registros de pitstops_df:")
    display(pitstops_df.head())

# Verificar si tenemos las columnas esperadas para laps
expected_laps_columns = [
    'LapTime', 'LapNumber', 'Stint', 'PitOutTime', 'PitInTime', 'Sector1Time', 
    'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 
    'Position', 'TyreLife', 'TrackStatus', 'IsAccurate', 'Compound', 'Driver'
]

# Verificar si tenemos las columnas esperadas para weather
expected_weather_columns = [
    'Time', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall', 
    'TrackTemp', 'WindDirection', 'WindSpeed'
]

# Verificar columnas esperadas para pitstops
expected_pitstop_columns = [
    'Time', 'Driver', 'LapNumber', 'PitInTime', 'Compound', 'TyreLife', 'FreshTyre'
]

# Verificar columnas disponibles en laps_df
print("\nColumnas disponibles en laps_df:")
for col in expected_laps_columns:
    if col in laps_df.columns:
        print(f"✓ {col}")
    else:
        print(f"✗ {col}")

# Verificar columnas disponibles en weather_df
print("\nColumnas disponibles en weather_df:")
for col in expected_weather_columns:
    if col in weather_df.columns:
        print(f"✓ {col}")
    else:
        print(f"✗ {col}")

# Verificar columnas disponibles en pitstops_df si existe
if has_pitstops:
    print("\nColumnas disponibles en pitstops_df:")
    for col in expected_pitstop_columns:
        if col in pitstops_df.columns:
            print(f"✓ {col}")
        else:
            print(f"✗ {col}")

## 5. Preprocesamiento y Unión de Datos

Unimos los datos de laps, weather, intervals y pitstops.

In [ ]:
def merge_all_data(laps_df, weather_df, intervals_df=None, pitstops_df=None):
    """
    Combina los DataFrames de laps, weather, intervals y pitstops
    """
    # Primero unir laps y weather basados en el tiempo
    if 'Time' in laps_df.columns and 'Time' in weather_df.columns:
        # Convertir a datetime si aún no lo está
        if not pd.api.types.is_datetime64_any_dtype(laps_df['Time']):
            laps_df['Time'] = pd.to_datetime(laps_df['Time'])
        if not pd.api.types.is_datetime64_any_dtype(weather_df['Time']):
            weather_df['Time'] = pd.to_datetime(weather_df['Time'])
            
        # Combinar datos basados en el tiempo más cercano
        merged_df = pd.merge_asof(
            laps_df.sort_values('Time'), 
            weather_df.sort_values('Time'),
            on='Time',
            direction='nearest'
        )
    else:
        print("Las columnas 'Time' no están disponibles en ambos DataFrames. Usando laps_df únicamente.")
        merged_df = laps_df.copy()
    
    # Si tenemos pitstops, agregar información de paradas
    if pitstops_df is not None:
        # Preparar datos de pitstops para unión
        print("Enriqueciendo datos con información de pitstops...")
        
        # 1. Calcular duración de pitstop para cada parada
        if 'PitInTime' in pitstops_df.columns and 'Time' in pitstops_df.columns:
            if not pd.api.types.is_datetime64_any_dtype(pitstops_df['PitInTime']):
                pitstops_df['PitInTime'] = pd.to_datetime(pitstops_df['PitInTime'])
            if not pd.api.types.is_datetime64_any_dtype(pitstops_df['Time']):
                pitstops_df['Time'] = pd.to_datetime(pitstops_df['Time'])
            
            # Tiempo en pit = Time (salida) - PitInTime (entrada)
            pitstops_df['PitDuration'] = (pitstops_df['Time'] - pitstops_df['PitInTime']).dt.total_seconds()
        
        # 2. Crear variables auxiliares para unir con laps
        pitstops_features = pitstops_df[['Driver', 'LapNumber', 'Compound', 'FreshTyre', 'PitDuration']].copy()
        pitstops_features.columns = ['Driver', 'PitStopLap', 'NextCompound', 'FreshTyreAfterStop', 'PitDuration']
        
        # 3. Unir pitstops con laps para indicar cuándo hay parada en la siguiente vuelta
        merged_df = pd.merge(
            merged_df,
            pitstops_features,
            left_on=['Driver', 'LapNumber'],
            right_on=['Driver', 'PitStopLap'],
            how='left'
        )
        
        # 4. Crear columna que indique si hay parada en la siguiente vuelta
        merged_df['PitNextLap'] = merged_df['PitDuration'].notna().astype(int)
        
        # 5. Crear columna con vueltas desde última parada
        def calculate_laps_since_pitstop(group):
            # Inicializar columna con valores altos
            group['LapsSincePitStop'] = 999
            
            # Obtener vueltas con parada
            pit_laps = group[group['PitNextLap'] == 1]['LapNumber'].values
            
            # Para cada vuelta, calcular distancia a la última parada
            for idx, row in group.iterrows():
                lap_num = row['LapNumber']
                # Encontrar la parada anterior más cercana
                previous_pits = [p for p in pit_laps if p < lap_num]
                if previous_pits:
                    group.loc[idx, 'LapsSincePitStop'] = lap_num - max(previous_pits)
                else:
                    # Si no hay parada previa, mantener en 999 o asignar otro valor
                    pass
            
            return group
        
        # Aplicar cálculo por piloto
        try:
            merged_df = merged_df.groupby('Driver').apply(calculate_laps_since_pitstop).reset_index(drop=True)
            
            # Vueltas desde la primera vuelta para el primer stint
            merged_df.loc[merged_df['LapsSincePitStop'] == 999, 'LapsSincePitStop'] = merged_df.loc[merged_df['LapsSincePitStop'] == 999, 'LapNumber']
        except Exception as e:
            print(f"Error al calcular LapsSincePitStop: {e}. Continuando sin esta característica.")
    
    # Si tenemos intervals, intentar unirlos también
    if intervals_df is not None:
        # Verificar columnas disponibles
        print("Columnas en intervals_df:", intervals_df.columns.tolist())
        
        # Intentar unir por driver_number y LapNumber si están disponibles
        if 'driver_number' in intervals_df.columns and 'DriverNumber' in merged_df.columns:
            print("Uniendo con intervals_df usando driver_number...")
            # Convertir a mismo tipo
            intervals_df['driver_number'] = intervals_df['driver_number'].astype(str)
            merged_df['DriverNumber'] = merged_df['DriverNumber'].astype(str)
            
            # Unir en driver_number
            merged_df = pd.merge(
                merged_df,
                intervals_df,
                left_on=['DriverNumber'],
                right_on=['driver_number'],
                how='left'
            )
        else:
            # Intentar encontrar otras columnas comunes
            common_cols = set(merged_df.columns).intersection(set(intervals_df.columns))
            if len(common_cols) > 0:
                print(f"Uniendo con intervals_df usando columna común: {list(common_cols)[0]}")
                
                join_col = list(common_cols)[0]
                merged_df = pd.merge(
                    merged_df,
                    intervals_df,
                    on=join_col,
                    how='left'
                )
            else:
                print("No se encontraron columnas comunes para unir con intervals_df.")
    
    return merged_df

# Unir todos los datos
merged_data = merge_all_data(
    laps_df, 
    weather_df, 
    intervals_df if has_intervals else None,
    pitstops_df if has_pitstops else None
)

# Verificar el resultado
print("Dimensiones del DataFrame combinado:", merged_data.shape)
display(merged_data.head())

## 6. Feature Engineering y Limpieza de Datos

In [ ]:
def prepare_data_for_modeling(df):
    """
    Realiza la limpieza y feature engineering en el DataFrame combinado
    """
    # Trabajar con una copia para no modificar el original
    data = df.copy()
    
    # Convertir LapTime a segundos (si es timedelta)
    if 'LapTime' in data.columns and pd.api.types.is_timedelta64_dtype(data['LapTime']):
        data['LapTime'] = data['LapTime'].dt.total_seconds()
        print("Convertida LapTime a segundos.")
    
    # Convertir tiempos de sector a segundos si están disponibles
    for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']:
        if sector in data.columns and pd.api.types.is_timedelta64_dtype(data[sector]):
            data[sector] = data[sector].dt.total_seconds()
            print(f"Convertido {sector} a segundos.")
    
    # Eliminar outliers en LapTime
    if 'LapTime' in data.columns:
        # Filtrar vueltas válidas (eliminar outliers)
        q1 = data['LapTime'].quantile(0.05)
        q3 = data['LapTime'].quantile(0.95)
        data = data[(data['LapTime'] >= q1) & (data['LapTime'] <= q3)]
        print(f"Eliminados outliers en LapTime. Rango válido: {q1:.2f}s - {q3:.2f}s")
    
    # Feature Engineering
    
    # 1. Edad de los neumáticos
    if 'TyreLife' in data.columns:
        data['TyreAge'] = data['TyreLife']
        print("Creada feature: TyreAge")
    
    # 2. Cambio de posición (comparado con la vuelta anterior)
    if 'Position' in data.columns and 'Driver' in data.columns:
        data['PositionChange'] = data.groupby('Driver')['Position'].diff().fillna(0)
        print("Creada feature: PositionChange")
    
    # 3. Carga de combustible (aproximación basada en la vuelta)
    if 'LapNumber' in data.columns:
        max_lap = data['LapNumber'].max()
        data['FuelLoad'] = 1 - (data['LapNumber'] / max_lap)  # Aproximación simple
        print("Creada feature: FuelLoad (aproximación)")
    
    # 4. DRS usado (si está disponible)
    if 'drs_window' in data.columns:
        data['DRSUsed'] = data['drs_window'].astype(int)
        print("Creada feature: DRSUsed")
    
    # 5. Ventana de undercut (si está disponible)
    if 'undercut_window' in data.columns:
        data['UndercutWindow'] = data['undercut_window'].astype(int)
        print("Creada feature: UndercutWindow")
    
    # 6. Gap al líder (si está disponible)
    if 'gap_to_leader_numeric' in data.columns:
        # Convertir a float por si acaso
        data['GapToLeader'] = pd.to_numeric(data['gap_to_leader_numeric'], errors='coerce')
        print("Creada feature: GapToLeader")
    
    # 7. Piloto en vuelta perdida (si está disponible)
    if 'is_lapped' in data.columns:
        data['IsLapped'] = data['is_lapped'].astype(int)
        print("Creada feature: IsLapped")
    
    # 8. Indicador de parada en la siguiente vuelta (si está disponible)
    if 'PitNextLap' in data.columns:
        print("Creada feature: PitNextLap (ya existente)")
    
    # 9. Vueltas desde última parada (si está disponible)
    if 'LapsSincePitStop' in data.columns:
        print("Creada feature: LapsSincePitStop (ya existente)")
    
    # 10. Duración de parada (si está disponible)
    if 'PitDuration' in data.columns:
        # Crear una columna con el tiempo perdido en pit
        # Para vueltas sin parada, el valor es 0
        data['PitTimeLost'] = data['PitDuration'].fillna(0)
        print("Creada feature: PitTimeLost")
    
    # 11. Cambio de compuesto en la próxima parada (si está disponible)
    if 'NextCompound' in data.columns and 'Compound' in data.columns:
        # Marcar cuando hay un cambio de compuesto (soft->medium, etc.)
        data['CompoundChange'] = (data['NextCompound'] != data['Compound']).astype(int)
        print("Creada feature: CompoundChange")
    
    # Asegurarse de tener el nombre correcto de la columna de compuesto
    if 'Compound' not in data.columns and 'TyreCompound' in data.columns:
        data['Compound'] = data['TyreCompound']
    
    # Seleccionar columnas relevantes para modelado
    features = [
        # Datos básicos de vuelta
        'LapNumber', 'Compound', 'TyreAge', 'Position', 'PositionChange', 'FuelLoad',
        # Velocidades (si están disponibles)
        'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST',
        # Clima
        'TrackTemp', 'AirTemp', 'Humidity', 'Pressure', 'WindSpeed',
        # Status
        'TrackStatus', 'IsAccurate',
        # Paradas
        'PitNextLap', 'LapsSincePitStop', 'PitTimeLost', 'CompoundChange',
        # Intervalos (si están disponibles)
        'DRSUsed', 'UndercutWindow', 'GapToLeader', 'IsLapped',
        # Driver
        'Driver'
    ]
    
    # Filtrar solo columnas que existen en el DataFrame
    available_features = [f for f in features if f in data.columns]
    print(f"\nCaracterísticas disponibles para modelado ({len(available_features)}):\n{available_features}")
    
    # Seleccionar solo filas con LapTime y features disponibles
    model_data = data[available_features + ['LapTime']].dropna(subset=['LapTime'])
    
    # Eliminar filas con NaN en features críticas
    critical_features = [f for f in ['Compound', 'LapNumber', 'TrackTemp'] if f in available_features]
    if critical_features:
        model_data = model_data.dropna(subset=critical_features)
    
    print(f"\nDimensiones finales del DataFrame para modelado: {model_data.shape}")
    
    return model_data, available_features

# Preparar datos para modelado
model_data, available_features = prepare_data_for_modeling(merged_data)

## 7. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Resumen estadístico
print("Resumen estadístico de variables numéricas:")
display(model_data.describe())

# Distribución de LapTime
plt.figure(figsize=(10, 6))
sns.histplot(model_data['LapTime'], kde=True)
plt.title('Distribución de Tiempos por Vuelta')
plt.xlabel('Tiempo (segundos)')
plt.ylabel('Frecuencia')
plt.savefig('outputs/week3/laptime_distribution.png')
plt.show()

# Tiempo por vuelta según tipo de neumático
if 'Compound' in model_data.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Compound', y='LapTime', data=model_data)
    plt.title('Tiempo por Vuelta según Tipo de Neumático')
    plt.xlabel('Compuesto')
    plt.ylabel('Tiempo (segundos)')
    plt.savefig('outputs/week3/laptime_by_tyre.png')
    plt.show()

# Evolución del tiempo por vuelta a lo largo de la carrera
if 'LapNumber' in model_data.columns and 'Driver' in model_data.columns:
    plt.figure(figsize=(14, 8))
    top_drivers = model_data['Driver'].value_counts().nlargest(5).index.tolist()
    for driver in top_drivers:
        driver_data = model_data[model_data['Driver'] == driver]
        plt.plot(driver_data['LapNumber'], driver_data['LapTime'], marker='o', linestyle='-', label=driver)
    plt.title('Evolución del Tiempo por Vuelta - Top 5 Pilotos')
    plt.xlabel('Número de Vuelta')
    plt.ylabel('Tiempo (segundos)')
    plt.legend()
    plt.grid(True)
    plt.savefig('outputs/week3/laptime_evolution.png')
    plt.show()

# Matriz de correlación
# Seleccionar solo columnas numéricas
numeric_data = model_data.select_dtypes(include=['int64', 'float64'])
# Limitar a 15 columnas para mejor visualización
if numeric_data.shape[1] > 15:
    # Incluir LapTime y las columnas más correlacionadas con ella
    if 'LapTime' in numeric_data.columns:
        corr_with_laptime = numeric_data.corr()['LapTime'].abs().sort_values(ascending=False)
        top_columns = corr_with_laptime.index[:14].tolist()  # 14 + LapTime = 15
        if 'LapTime' not in top_columns:  # Por si acaso
            top_columns = ['LapTime'] + top_columns[:13]
        numeric_data = numeric_data[top_columns]
    else:
        numeric_data = numeric_data.iloc[:, :15]  # Primeras 15 columnas

correlation_matrix = numeric_data.corr()

# Filtrar correlaciones con LapTime
if 'LapTime' in correlation_matrix.columns:
    laptime_correlations = correlation_matrix['LapTime'].sort_values(ascending=False)
    print("\nCorrelaciones con LapTime:")
    print(laptime_correlations)

# Visualizar matriz de correlación
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt=".2f")
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.savefig('outputs/week3/correlation_matrix.png')
plt.show()

# Diagrama de dispersión: Variables importantes vs LapTime
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

if 'TyreAge' in model_data.columns:
    sns.scatterplot(x='TyreAge', y='LapTime', hue='Compound', data=model_data, ax=axes[0])
    axes[0].set_title('LapTime vs TyreAge')
    
if 'TrackTemp' in model_data.columns:
    sns.scatterplot(x='TrackTemp', y='LapTime', data=model_data, ax=axes[1])
    axes[1].set_title('LapTime vs TrackTemp')
    
if 'FuelLoad' in model_data.columns:
    sns.scatterplot(x='FuelLoad', y='LapTime', data=model_data, ax=axes[2])
    axes[2].set_title('LapTime vs FuelLoad')

if 'Position' in model_data.columns:
    sns.scatterplot(x='Position', y='LapTime', data=model_data, ax=axes[3])
    axes[3].set_title('LapTime vs Position')
    
plt.tight_layout()
plt.savefig('outputs/week3/key_variables_scatter.png')
plt.show()

## 7.1 Análisis Específico de Paradas en Boxes

In [ ]:
def visualize_pitstop_impact(model_data):
    """
    Visualiza el impacto de las paradas en los tiempos por vuelta
    """
    if 'LapsSincePitStop' not in model_data.columns:
        print("No hay datos de paradas disponibles para visualización.")
        return
    
    # 1. Efecto de la edad de los neumáticos en el tiempo por vuelta
    plt.figure(figsize=(14, 8))
    
    # Filtrar para mostrar hasta 20 vueltas después de una parada
    plot_data = model_data[model_data['LapsSincePitStop'] <= 20].copy()
    
    # Agrupar por tipo de compuesto y vueltas desde parada
    plot_data['LapsSincePitStop'] = plot_data['LapsSincePitStop'].astype(int)
    grouped = plot_data.groupby(['Compound', 'LapsSincePitStop'])['LapTime'].mean().reset_index()
    
    # Dibujar una línea por cada compuesto
    for compound in grouped['Compound'].unique():
        compound_data = grouped[grouped['Compound'] == compound]
        plt.plot(compound_data['LapsSincePitStop'], 
                 compound_data['LapTime'], 
                 'o-', 
                 label=f'Compuesto {compound}')
    
    plt.xlabel('Vueltas desde la parada')
    plt.ylabel('Tiempo por vuelta promedio (s)')
    plt.title('Degradación de neumáticos por compuesto')
    plt.legend()
    plt.grid(True)
    plt.savefig('outputs/week3/tyre_degradation.png')
    plt.show()
    
    # 2. Proporción de paradas por compuesto
    if 'PitNextLap' in model_data.columns and model_data['PitNextLap'].sum() > 0:
        pitstop_data = model_data[model_data['PitNextLap'] == 1]
        
        plt.figure(figsize=(10, 6))
        sns.countplot(x='Compound', data=pitstop_data)
        plt.title('Compuestos utilizados antes de paradas')
        plt.xlabel('Compuesto')
        plt.ylabel('Número de paradas')
        plt.savefig('outputs/week3/compounds_before_pitstop.png')
        plt.show()
        
        # 3. Distribución de duración de paradas
        if 'PitTimeLost' in model_data.columns:
            pitstop_times = model_data[model_data['PitTimeLost'] > 0]['PitTimeLost']
            
            if not pitstop_times.empty:
                plt.figure(figsize=(10, 6))
                sns.histplot(pitstop_times, kde=True, bins=15)
                plt.axvline(pitstop_times.mean(), color='r', linestyle='--', 
                           label=f'Media: {pitstop_times.mean():.2f}s')
                plt.title('Distribución de tiempo perdido en paradas')
                plt.xlabel('Tiempo (s)')
                plt.ylabel('Frecuencia')
                plt.legend()
                plt.savefig('outputs/week3/pitstop_duration.png')
                plt.show()
    
    # 4. Comparación de tiempos antes/después de paradas
    if 'PitNextLap' in model_data.columns and 'LapsSincePitStop' in model_data.columns:
        # Obtener vueltas justo antes de una parada
        pre_stop_laps = model_data[model_data['PitNextLap'] == 1].copy()
        if not pre_stop_laps.empty:
            pre_stop_laps['LapType'] = 'Pre-Stop'
            
            # Obtener primera vuelta después de cada parada (LapsSincePitStop = 1)
            post_stop_laps = model_data[model_data['LapsSincePitStop'] == 1].copy()
            if not post_stop_laps.empty:
                post_stop_laps['LapType'] = 'Post-Stop'
                
                # Combinar datos
                comparison_df = pd.concat([pre_stop_laps, post_stop_laps])
                
                # Visualizar
                plt.figure(figsize=(10, 6))
                sns.boxplot(x='Compound', y='LapTime', hue='LapType', data=comparison_df)
                plt.title('Comparación de tiempos antes y después de paradas')
                plt.xlabel('Compuesto')
                plt.ylabel('Tiempo por vuelta (s)')
                plt.savefig('outputs/week3/pre_post_stop_comparison.png')
                plt.show()

# Llamar a la función de visualización
if has_pitstops:
    print("\nAnalizando impacto de paradas en tiempos por vuelta...")
    visualize_pitstop_impact(model_data)

## 8. Preprocesamiento para Modelado

In [ ]:
def preprocess_data_for_modeling(df, features):
    """
    Preprocesa los datos para el modelado
    """
    # Separar características y objetivo
    X = df[features]
    y = df['LapTime']
    
    # Identificar columnas categóricas y numéricas
    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Mostrar información sobre las columnas
    print(f"Características categóricas ({len(cat_cols)}): {cat_cols}")
    print(f"Características numéricas ({len(num_cols)}): {num_cols}")
    
    # Crear preprocesadores
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    # Combinar preprocesadores
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)
        ])
    
    # Dividir datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    
    print(f"Tamaño de conjunto de entrenamiento: {X_train.shape}")
    print(f"Tamaño de conjunto de prueba: {X_test.shape}")
    
    return X_train, X_test, y_train, y_test, preprocessor

# Preprocesar datos para modelado
X_train, X_test, y_train, y_test, preprocessor = preprocess_data_for_modeling(model_data, available_features)

## 9. Entrenamiento de Modelo XGBoost

In [ ]:
def train_xgboost(X_train, X_test, y_train, y_test, preprocessor):
    """
    Entrena un modelo XGBoost
    """
    # Crear pipeline con preprocesamiento y modelo
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(objective='reg:squarederror'))
    ])
    
    # Parámetros para Grid Search
    param_grid = {
        'regressor__n_estimators': [100, 200],
        'regressor__learning_rate': [0.01, 0.1],
        'regressor__max_depth': [3, 5, 7],
        'regressor__min_child_weight': [1, 3]
    }
    
    # Grid Search
    grid_search = GridSearchCV(
        model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1
    )
    
    # Entrenar modelo
    print("Entrenando modelo XGBoost con GridSearchCV (esto puede tardar varios minutos)...")
    grid_search.fit(X_train, y_train)
    
    # Mejores parámetros
    print("\nMejores parámetros XGBoost:")
    for param, value in grid_search.best_params_.items():
        print(f"  {param}: {value}")
    
    # Predecir
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    # Evaluar
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nMétricas de evaluación del modelo XGBoost:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f} segundos")
    print(f"  MAE: {mae:.4f} segundos")
    print(f"  R²: {r2:.4f}")
    
    # Analizar características importantes
    if hasattr(best_model.named_steps['regressor'], 'feature_importances_'):
        # Obtener nombres de características después del preprocesamiento
        try:
            cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
            num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
            
            # Obtener nombres de características después de one-hot encoding
            encoder = best_model.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot']
            cat_features = encoder.get_feature_names_out(cat_cols).tolist()
            all_features = num_cols + cat_features
            
            # Crear DataFrame con importancias
            importances = best_model.named_steps['regressor'].feature_importances_
            feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importances})
            feature_importance = feature_importance.sort_values('Importance', ascending=False).head(15)
            
            # Visualizar
            plt.figure(figsize=(12, 8))
            sns.barplot(x='Importance', y='Feature', data=feature_importance)
            plt.title('Top 15 Características Más Importantes - XGBoost')
            plt.tight_layout()
            plt.savefig('outputs/week3/xgboost_feature_importance.png')
            plt.show()
        except Exception as e:
            print(f"Error al analizar importancia de características: {e}")
    
    return best_model, y_pred

# Entrenar modelo XGBoost
xgb_model, y_pred_xgb = train_xgboost(X_train, X_test, y_train, y_test, preprocessor)

## 10. Entrenamiento de Red Neuronal (Opcional)

Este paso es opcional y puede omitirse si prefieres usar solo XGBoost.

In [ ]:
def train_pytorch_nn(X_train, X_test, y_train, y_test, preprocessor):
    """
    Entrena un modelo de red neuronal con PyTorch
    """
    # Aplicar preprocesamiento
    print("Preprocesando datos para la red neuronal...")
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # Convertir a tensores
    X_train_tensor = torch.FloatTensor(X_train_processed.toarray())
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test_processed.toarray())
    y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)
    
    # Crear conjuntos de datos y cargadores
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    # Inicializar modelo
    print(f"Inicializando red neuronal con {X_train_processed.shape[1]} entradas...")
    input_size = X_train_processed.shape[1]
    model = LapTimeNN(input_size)
    
    # Definir criterio y optimizador
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Guardar historial de pérdidas para graficar
    losses = []
    
    # Entrenar modelo
    print("\nEntrenando red neuronal...")
    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)
        
        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Visualizar curva de aprendizaje
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs+1), losses)
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title('Curva de Aprendizaje - Red Neuronal')
    plt.grid(True)
    plt.savefig('outputs/week3/neural_network_learning_curve.png')
    plt.show()
    
    # Evaluar modelo
    model.eval()
    with torch.no_grad():
        y_pred_tensor = model(X_test_tensor)
        y_pred = y_pred_tensor.numpy().flatten()
        
    # Métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nMétricas de evaluación de la Red Neuronal:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f} segundos")
    print(f"  MAE: {mae:.4f} segundos")
    print(f"  R²: {r2:.4f}")
    
    return model, y_pred

# Preguntar si se desea entrenar la red neuronal
train_nn = True  # Cambiar a False para omitir este paso
if train_nn:
    nn_model, y_pred_nn = train_pytorch_nn(X_train, X_test, y_train, y_test, preprocessor)
else:
    nn_model, y_pred_nn = None, None

## 11. Visualización y Comparación de Resultados

In [ ]:
def visualize_predictions(y_test, y_pred_xgb, y_pred_nn=None, max_points=1000):
    """
    Visualiza los resultados de los modelos
    """
    # Limitar número de puntos para visualización clara
    if len(y_test) > max_points:
        # Muestrear aleatoriamente para mantener la distribución
        indices = np.random.choice(len(y_test), max_points, replace=False)
        y_test_sample = y_test.iloc[indices]
        y_pred_xgb_sample = y_pred_xgb[indices]
        if y_pred_nn is not None:
            y_pred_nn_sample = y_pred_nn[indices]
    else:
        y_test_sample = y_test
        y_pred_xgb_sample = y_pred_xgb
        y_pred_nn_sample = y_pred_nn
    
    # Configurar tamaño de figura
    plt.figure(figsize=(16, 8))
    
    # Determinar número de subplots
    if y_pred_nn is not None:
        n_plots = 3
    else:
        n_plots = 2
    
    # 1. Dispersión XGBoost
    plt.subplot(1, n_plots, 1)
    plt.scatter(y_test_sample, y_pred_xgb_sample, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Tiempo real (s)')
    plt.ylabel('Tiempo predicho (s)')
    plt.title('XGBoost: Predicciones vs Reales')
    
    # 2. Residuos XGBoost
    plt.subplot(1, n_plots, 2)
    residuals_xgb = y_test_sample - y_pred_xgb_sample
    plt.scatter(y_pred_xgb_sample, residuals_xgb, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicción (s)')
    plt.ylabel('Residuos (s)')
    plt.title('XGBoost: Residuos vs Predicciones')
    
    # 3. Si hay predicciones de red neuronal
    if y_pred_nn is not None:
        plt.subplot(1, n_plots, 3)
        plt.scatter(y_test_sample, y_pred_nn_sample, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        plt.xlabel('Tiempo real (s)')
        plt.ylabel('Tiempo predicho (s)')
        plt.title('Red Neuronal: Predicciones vs Reales')
    
    plt.tight_layout()
    plt.savefig('outputs/week3/prediction_results.png')
    plt.show()
    
    # Comparativa de errores
    if y_pred_nn is not None:
        errors_xgb = np.abs(y_test - y_pred_xgb)
        errors_nn = np.abs(y_test - y_pred_nn)
        
        plt.figure(figsize=(12, 6))
        plt.hist(errors_xgb, alpha=0.5, bins=50, label='XGBoost')
        plt.hist(errors_nn, alpha=0.5, bins=50, label='Red Neuronal')
        plt.legend()
        plt.xlabel('Error Absoluto (s)')
        plt.ylabel('Frecuencia')
        plt.title('Distribución de Errores por Modelo')
        plt.xlim(0, min(10, max(errors_xgb.max(), errors_nn.max())))
        plt.savefig('outputs/week3/error_distribution.png')
        plt.show()

# Visualizar resultados
visualize_predictions(y_test, y_pred_xgb, y_pred_nn if train_nn else None)

## 12. Guardar Modelos

In [ ]:
def save_models(xgb_model, nn_model=None):
    """
    Guarda los modelos entrenados
    """
    # Guardar modelo XGBoost
    joblib.dump(xgb_model, 'models/week3/xgboost_laptime.joblib')
    print("Modelo XGBoost guardado en 'models/week3/xgboost_laptime.joblib'")
    
    # Guardar modelo PyTorch si existe
    if nn_model is not None:
        torch.save(nn_model.state_dict(), 'models/week3/nn_laptime.pth')
        print("Modelo PyTorch guardado en 'models/week3/nn_laptime.pth'")

# Guardar modelos
save_models(xgb_model, nn_model if train_nn else None)

## 13. Prueba de Predicción con Nuevos Datos

Probemos el modelo con una situación de carrera hipotética.

In [ ]:
def test_prediction(model, available_features):
    """
    Prueba el modelo con datos hipotéticos
    """
    # Crear un escenario de ejemplo
    example_drivers = ["VER", "HAM", "LEC", "PER", "SAI"]
    compounds = ["SOFT", "MEDIUM", "HARD"]
    
    # Crear DataFrame de ejemplo
    example_data = []
    
    for driver in example_drivers:
        for compound in compounds:
            for tyre_age in [1, 10, 20]:
                # Valores predeterminados
                row = {
                    'Driver': driver,
                    'Compound': compound,
                    'TyreAge': tyre_age,
                    'LapNumber': 30,
                    'TrackTemp': 35.0,
                    'AirTemp': 25.0,
                    'Position': 5,
                    'FuelLoad': 0.5,
                    'LapsSincePitStop': tyre_age,  # Si está disponible
                    'PitNextLap': 0,  # No hay parada en la siguiente vuelta
                }
                
                # Añadir solo características disponibles
                example_row = {k: v for k, v in row.items() if k in available_features}
                example_data.append(example_row)
    
    example_df = pd.DataFrame(example_data)
    
    # Hacer predicciones
    predictions = model.predict(example_df)
    
    # Añadir predicciones al DataFrame
    example_df['PredictedLapTime'] = predictions
    
    # Visualizar resultados
    print("\nPredicciones para escenarios de ejemplo:")
    display(example_df)
    
    # Graficar por compuesto y edad de neumáticos
    if 'Compound' in example_df.columns and 'TyreAge' in example_df.columns:
        plt.figure(figsize=(12, 8))
        for driver in example_drivers[:3]:  # Limitar a 3 pilotos para claridad
            driver_data = example_df[example_df['Driver'] == driver]
            for compound in compounds:
                tyre_data = driver_data[driver_data['Compound'] == compound]
                plt.plot(tyre_data['TyreAge'], tyre_data['PredictedLapTime'], 
                         marker='o', linestyle='-', label=f"{driver} - {compound}")
                
        plt.xlabel('Edad de Neumáticos (vueltas)')
        plt.ylabel('Tiempo Predicho (s)')
        plt.title('Predicción de Tiempos por Tipo y Edad de Neumáticos')
        plt.legend()
        plt.grid(True)
        plt.savefig('outputs/week3/tyre_age_predictions.png')
        plt.show()
    
    return example_df

# Probar predicciones con el modelo XGBoost
try:
    prediction_examples = test_prediction(xgb_model, available_features)
except Exception as e:
    print(f"Error al realizar predicciones de prueba: {e}")

## 14. Conclusiones y Próximos Pasos

### Conclusiones

En este notebook, hemos:
1. Cargado y preparado datos de FastF1 y OpenF1, incluyendo información de vueltas, paradas y condiciones meteorológicas
2. Realizado feature engineering para crear características como edad de neumáticos, carga de combustible, y paradas en boxes
3. Analizado el impacto de las paradas y la degradación de neumáticos en los tiempos por vuelta
4. Entrenado un modelo XGBoost para predecir tiempos por vuelta
5. Opcionalmente, entrenado una red neuronal para comparar su rendimiento
6. Visualizado y evaluado los resultados

### Resultados Principales
- El modelo XGBoost puede predecir tiempos por vuelta con un error medio de X segundos
- Las características más importantes para la predicción son [listar las top 3-5 características]
- La degradación de neumáticos tiene un impacto significativo en los tiempos por vuelta
- Las paradas en boxes muestran patrones claros en cuanto a su timing y elección de compuesto

### Próximos Pasos (Semana 4)
1. Integrar este modelo de predicción con un sistema de decisiones basado en reglas
2. Añadir lógica para recomendar estrategias de paradas en boxes
3. Desarrollar sistema para simular undercuts/overcuts basados en las predicciones
4. Crear visualizaciones interactivas para analizar el impacto de diferentes estrategias